## Demo - BandMaths

In [1]:
from snapista import Graph
from snapista import Operator
from snapista import read_file
from snapista import TargetBand
from snapista import TargetBandDescriptors

#import os 
#os.environ['PATH'] = '/srv/conda/envs/env_snap/snap/bin:' +  os.environ['PATH']

In [4]:

try:
    g = Graph()
except Exception:
    # running in vs container
    import os 
    os.environ['PATH'] = '/srv/conda/envs/env_snap/snap/bin:' +  os.environ['PATH']
    
    g = Graph()

In [5]:
g.add_node(operator=Operator("Read", file='/data/S2B_MSIL2A_20210608T112119_N0300_R037_T29SMC_20210608T131325.SAFE/MTD_MSIL2A.xml'), node_id="Read")


In [6]:
g.add_node(operator=Operator("Resample", referenceBandName="B2"), node_id="Resample", source="Read")

In [7]:
band_maths = Operator('BandMaths')

cloud_class = TargetBand(name='Cloud_Class',
                              expression='if quality_cloud_confidence > 90 then 2 else (if quality_cloud_confidence > 50 AND B8 > 0.3 then 1 else 0)')
                         
band_maths.targetBandDescriptors = TargetBandDescriptors([cloud_class])

In [8]:
g.add_node(operator=band_maths, node_id="BandMaths" ,source='Resample')

targetBandDescriptors <snapista.target_band_descriptors.TargetBandDescriptors object at 0x7fd2f06080a0>


In [9]:
band_maths = Operator('BandMaths')

ndsi = TargetBand(name='NDSI',
                              expression='(B3-B11)/(B3+B11)')
                         
band_maths.targetBandDescriptors = TargetBandDescriptors([ndsi])

g.add_node(operator=band_maths, node_id="BandMaths_2" ,source='Resample')

targetBandDescriptors <snapista.target_band_descriptors.TargetBandDescriptors object at 0x7fd2f05eca60>


In [10]:
merge = Operator("BandMerge", sourceBandNames="B3,B4,B11,B12,Cloud_Class,NDSI", source=["Resample", "BandMaths", 'BandMaths_2'])

merge

Operator('BandMerge', sourceBandNames='B3,B4,B11,B12,Cloud_Class,NDSI', geographicError='1.0E-5f', source='['Resample', 'BandMaths', 'BandMaths_2']')

In [11]:
g.add_node(operator=merge, node_id="Merge", source=["Resample", "BandMaths", 'BandMaths_2'])

In [12]:
write = Operator("Write")

write

Operator('Write', file='None', formatName='BEAM-DIMAP', deleteOutputOnFailure='true', writeEntireTileRows='false', clearCacheAfterRowWrite='false')

In [13]:
g.add_node(operator=Operator("Write", file='snow.dim'), node_id="Write", source='Merge')

In [14]:
g.view()

<graph>
  <version>1.0</version>
  <node id="Read">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <bandNames/>
      <copyMetadata>true</copyMetadata>
      <file>/data/S2B_MSIL2A_20210608T112119_N0300_R037_T29SMC_20210608T131325.SAFE/MTD_MSIL2A.xml</file>
      <formatName/>
      <geometryRegion/>
      <maskNames/>
      <pixelRegion/>
    </parameters>
  </node>
  <node id="Resample">
    <operator>Resample</operator>
    <sources>
      <sourceProduct refid="Read"/>
    </sources>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <bandResamplings/>
      <downsamplingMethod>First</downsamplingMethod>
      <flagDownsamplingMethod>First</flagDownsamplingMethod>
      <referenceBandName>B2</referenceBandName>
      <resampleOnPyramidLevels>true</resampleOnPyramidLevels>
      <resamplingPreset/>
      <targetHeight/>
      <targetResolution/>
      <targetWidth/>
      <upsamplingMethod>Nearest<

In [15]:
g.save_graph('snow.xml')
g.run()

Processing the graph
Executing processing graph
INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
....10%....20%....30%....40%....50%....60%....70%....80%....90% done.
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: GDAL not found on system. Internal GDAL 3.0.0 from distribution will be used. (f1)
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.


0